In [18]:
import requests
import pandas as pd
import numpy as np
from google.colab import files
import os
import json
import zipfile

In [2]:
API_KEY = 'ixJc2ZRKUZ9MrYT4biffXa35ZcqdhxbtnAtbm3MJbkGWyesmYpMuY1laKHdvT9Sc'
HEADERS = {'X-TBA-Auth-Key': API_KEY}

In [3]:
district = 'fim'
year_events = {}
for year in range(2017, 2026):
    district_key = str(year) + district
    url = f'https://www.thebluealliance.com/api/v3/district/{district_key}/events/keys?year={year}'
    response = requests.get(url, headers=HEADERS)
    json = response.json()
    year_events.update({str(year): json})
year_events

{'2017': ['2017miann',
  '2017mibro',
  '2017micen',
  '2017micmp',
  '2017micmp1',
  '2017micmp2',
  '2017micmp3',
  '2017micmp4',
  '2017miesc',
  '2017migay',
  '2017migul',
  '2017mihow',
  '2017mike2',
  '2017miken',
  '2017miket',
  '2017milak',
  '2017milan',
  '2017miliv',
  '2017milsu',
  '2017mimar',
  '2017mimid',
  '2017mishe',
  '2017misjo',
  '2017misou',
  '2017mitry',
  '2017mitvc',
  '2017miwat',
  '2017miwmi'],
 '2018': ['2018mialp',
  '2018mibel',
  '2018micen',
  '2018micmp',
  '2018micmp1',
  '2018micmp2',
  '2018micmp3',
  '2018micmp4',
  '2018miesc',
  '2018mifor',
  '2018migay',
  '2018migib',
  '2018migul',
  '2018mike2',
  '2018miken',
  '2018miket',
  '2018milak',
  '2018milan',
  '2018milin',
  '2018miliv',
  '2018milsu',
  '2018mimar',
  '2018mimid',
  '2018mimil',
  '2018mishe',
  '2018misjo',
  '2018misou',
  '2018mitry',
  '2018mitvc',
  '2018miwat',
  '2018miwmi'],
 '2019': ['2019mial2',
  '2019mialp',
  '2019mibel',
  '2019micen',
  '2019micmp',
  '201

In [4]:
#each year -> each event -> matches and teams
#district points, total points, average score

year_matches = {}
year_teams = {}
year_rankings = {}
year_district_points = {}

for year, events in year_events.items():
  year_matches[year] = {}
  year_teams[year] = {}
  for event in events:
    match_url = f'https://www.thebluealliance.com/api/v3/event/{event}/matches'
    response = requests.get(match_url, headers=HEADERS)
    matches = response.json()
    num_matches = len(matches)

    teams_url = f'https://www.thebluealliance.com/api/v3/event/{event}/teams'
    response = requests.get(teams_url, headers=HEADERS)
    teams = response.json()

    year_matches[year][event] = matches
    year_teams[year][event] = teams

In [29]:
def match_list(team_key, event_key):
  year = event_key[:4]
  event_matches = year_matches[year][event_key]
  team_matches = []
  for match in event_matches:
    red_alliance = match['alliances']['red']['team_keys']
    blue_alliance = match['alliances']['blue']['team_keys']

    if team_key in red_alliance or team_key in blue_alliance :
      team_matches.append(match)
  return team_matches

def avg_score(team_key, event_key):
  matches = match_list(team_key, event_key)
  total = 0
  num = 0
  for match in matches:
    red_alliance = match['alliances']['red']['team_keys']
    blue_alliance = match['alliances']['blue']['team_keys']

    if team_key in red_alliance:
        total += match['alliances']['red']['score']
        num += 1
    elif team_key in blue_alliance:
        total += match['alliances']['blue']['score']
        num += 1
  return total/num

def avg_teleop(team_key, event_key):
  matches = match_list(team_key, event_key)
  total = 0
  num = 0
  for match in matches:
    red_alliance = match['alliances']['red']['team_keys']
    blue_alliance = match['alliances']['blue']['team_keys']

    if team_key in red_alliance:
        total += match['score_breakdown']['red']['teleopPoints']
        num += 1
    elif team_key in blue_alliance:
        total += match['score_breakdown']['blue']['teleopPoints']
        num += 1
  return total/num

def avg_auto(team_key, event_key):
  matches = match_list(team_key, event_key)
  total = 0
  num = 0
  for match in matches:
    red_alliance = match['alliances']['red']['team_keys']
    blue_alliance = match['alliances']['blue']['team_keys']

    if team_key in red_alliance:
        total += match['score_breakdown']['red']['autoPoints']
        num += 1
    elif team_key in blue_alliance:
        total += match['score_breakdown']['blue']['autoPoints']
        num += 1
  return total/num


def point_spread_avg(team_key, event_key):
  matches = match_list(team_key, event_key)
  spreads= []
  for match in matches:
    red_alliance = match['alliances']['red']['team_keys']
    blue_alliance = match['alliances']['blue']['team_keys']
    red_score = match['alliances']['red']['score']
    blue_score = match['alliances']['blue']['score']

    if team_key in red_alliance:
        diff = red_score - blue_score
        spreads.append(diff)
    elif team_key in blue_alliance:
        diff = blue_score - red_score
        spreads.append(diff)
  return np.mean(spreads)

def point_spread_std(team_key, event_key):
  matches = match_list(team_key, event_key)
  spreads= []
  for match in matches:
    red_alliance = match['alliances']['red']['team_keys']
    blue_alliance = match['alliances']['blue']['team_keys']
    red_score = match['alliances']['red']['score']
    blue_score = match['alliances']['blue']['score']

    if team_key in red_alliance:
        diff = red_score - blue_score
        spreads.append(diff)
    elif team_key in blue_alliance:
        diff = blue_score - red_score
        spreads.append(diff)
  return np.std(spreads)

In [30]:
match_list('frc67', '2025mimil')
print(avg_score('frc67', '2025mimil'))
print(avg_teleop('frc67', '2025mimil'))
print(avg_auto('frc67', '2025mimil'))

print(point_spread_avg('frc67', '2025mimil'))
print(point_spread_std('frc67', '2025mimil'))

124.61111111111111
91.22222222222223
28.944444444444443
37.72222222222222
32.0308975063411


In [19]:
output_base = 'frc_data_export'
zip_filename = 'frc_export.zip'

for year in year_matches:
    for event_key in year_matches[year]:
        # Define directory path: frc_data_export/2025/2025casf/
        folder_path = os.path.join(output_base, year, event_key)
        os.makedirs(folder_path, exist_ok=True)

        # Write matches.json
        with open(os.path.join(folder_path, 'matches.json'), 'w') as f:
            json.dump(year_matches[year][event_key], f, indent=2)

        # Write teams.json
        with open(os.path.join(folder_path, 'teams.json'), 'w') as f:
            json.dump(year_teams[year][event_key], f, indent=2)

# Step 2: Zip the whole output folder
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for foldername, subfolders, filenames in os.walk(output_base):
        for filename in filenames:
            filepath = os.path.join(foldername, filename)
            arcname = os.path.relpath(filepath, output_base)
            zipf.write(filepath, arcname)

print(f"✅ Zipped export created: {zip_filename}")
files.download('frc_export.zip')

✅ Zipped export created: frc_export.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
def event_point_spread_df(event_key, year_matches, year_teams):
    year = event_key[:4]
    teams = year_teams[year][event_key]

    data = []
    for team in teams:
        team_key = team['key']  # e.g., 'frc114'
        spread = point_spread_avg(team_key, event_key)
        data.append({'team': team_key, 'point_spread_avg': spread, 'rank' : rank})


    df = pd.DataFrame(data)
    df['rank'] = df['point_spread_avg'].rank(ascending=False, method='min').astype(int)
    df = df.sort_values(by='rank')
    return df

In [37]:
event_point_spread_df('2025mimil', year_matches, year_teams)

,team,point_spread_avg,rank
21,frc67,37.722222,1
8,frc3656,35.722222,2
24,frc7166,23.631579,3
7,frc3322,17.733333,4
0,frc1023,15.842105,5
5,frc2834,10.125000,6
18,frc6344,6.111111,7
17,frc6120,3.529412,8
28,frc7225,2.833333,9
30,frc7660,2.105263,10
